In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def scrape_product(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    }

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Note: Selectors vary by site. These are generic placeholders.
            title = soup.find('h1').get_text(strip=True) if soup.find('h1') else "N/A"
            price = soup.find('span', class_='price').get_text(strip=True) if soup.find('span', class_='price') else "N/A"

            data = {
                "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "Product Name": title,
                "Price": price,
                "Source URL": url
            }
            return data
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

# List of URLs to track
urls = ["https://www.limelight.pk/collections/embroidered-pret"]
results = [scrape_product(u) for u in urls if scrape_product(u)]

# Save to Excel
df = pd.DataFrame(results)
df.to_excel("price_tracking_results.xlsx", index=False)
print("Data saved to price_tracking_results.xlsx")

Data saved to price_tracking_results.xlsx


In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def scrape_product(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    }

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # 1. Capture the Product Name
            title = soup.find('h1').get_text(strip=True) if soup.find('h1') else "N/A"

            # 2. Capture the Price using the 'money' class from your inspect tool
            # We look for the span with class 'money' inside the price container
            price_elem = soup.select_one("span.price-item--regular span.money")

            # If it's a sale item, the class might change, so we add a fallback
            if not price_elem:
                price_elem = soup.select_one("span.price-item--sale span.money")

            price = price_elem.get_text(strip=True) if price_elem else "N/A"

            return {
                "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "Product Name": title,
                "Price": price,
                "Source URL": url
            }
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

# Your existing logic to run and save to Excel follows...
# List of URLs to track
urls = ["https://www.limelight.pk/collections/embroidered-pret"]
results = [scrape_product(u) for u in urls if scrape_product(u)]

# Save to Excel
df = pd.DataFrame(results)
df.to_excel("tracking_results.xlsx", index=False)
print("Data saved to tracking_results.xlsx")

Data saved to tracking_results.xlsx


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time

def scrape_limelight_page(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
    }

    product_list = []

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Step 1: Find all product cards (containers)
            # On Limelight, each product is usually in a 'div' or 'li' with a specific class
            products = soup.find_all('div', class_='product-card-wrapper')

            for product in products:
                # Step 2: Extract details for EACH product in the loop
                # Adjust these selectors based on your specific inspect tool findings
                name_elem = product.find('a', class_='full-unstyled-link')
                price_elem = product.select_one("span.price-item--regular span.money")

                if not price_elem:
                    price_elem = product.select_one("span.price-item--sale span.money")

                product_list.append({
                    "Timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    "Product Name": name_elem.get_text(strip=True) if name_elem else "N/A",
                    "Price": price_elem.get_text(strip=True) if price_elem else "N/A",
                    "Source URL": url
                })

            return product_list
    except Exception as e:
        print(f"Error scraping: {e}")
        return []

# Run the scraper
target_url = "https://www.limelight.pk/collections/embroidered-pret"
all_products = scrape_limelight_page(target_url)

# Save all results to Excel
if all_products:
    df = pd.DataFrame(all_products)
    df.to_excel("limelight_products.xlsx", index=False)
    print(f"Success! Saved {len(all_products)} products to Excel.")

Success! Saved 24 products to Excel.
